In [10]:
import os
import cv2
import numpy as np


#splitting data into Train/Test

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

**Tensorflow Keras CNN Model**

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten, Activation, Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta,RMSprop


In [19]:
import matplotlib.pyplot as plt
folder_dir='flowers'

In [ ]:
import os
import cv2

data = []
label = []

SIZE = 128  

for folder in os.listdir(folder_dir):
    folder_path = os.path.join(folder_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    for file in os.listdir(folder_path):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_path, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE, SIZE))
            data.append(im)
        else:
            continue


In [18]:
data_arr=np.array(data)
label_arr=np.array(label)
encoder=LabelEncoder()
y=encoder.fit_transform(label_arr)
y=to_categorical(y,5)
X=data_arr/25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [19]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (SIZE,SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5, activation = "softmax"))

In [20]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=32
epochs=64
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),
                              verbose = 1)

<ipython-input-21-ed2bcf72189a>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/64
21/21 [==============================] - 188s 9s/step - loss: 2.6406 - accuracy: 0.8587 - val_loss: 0.2906 - val_accuracy: 0.9321
Epoch 2/64
21/21 [==============================] - 190s 9s/step - loss: 0.4204 - accuracy: 0.8789 - val_loss: 0.2302 - val_accuracy: 0.9321
Epoch 3/64
21/21 [==============================] - 191s 9s/step - loss: 0.4074 - accuracy: 0.8991 - val_loss: 0.1971 - val_accuracy: 0.9321
Epoch 4/64
21/21 [==============================] - 174s 8s/step - loss: 0.2825 - accuracy: 0.9022 - val_loss: 0.3084 - val_accuracy: 0.9321
Epoch 5/64
21/21 [==============================] - 175s 8s/step - loss: 0.3610 - accuracy: 0.9037 - val_loss: 0.1882 - val_accuracy: 0.9321
Epoch 6/64
21/21 [==============================] - 185s 9s/step - loss: 0.2882 - accuracy: 0.9022 - val_loss: 0.2102 - val_accuracy: 0.9321
Epoch 7/64
21/21 [==============================] - 179s 9s/step - loss: 0.2505 - accuracy: 0.9037 - val_loss: 0.2688 - val_accuracy: 0.9321
Epoch 8/64
21

In [ ]:
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')